In [2]:
import warnings

import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from scipy.sparse import coo_matrix
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")

In [5]:
# Получение данных для коллаборативной фильтрации
events_raw = pd.read_csv("./data/events.csv")

In [3]:
# Основная функция рекомендаций
def rs_batch(data, lb_interactions, upb_ineractions, top_items):
    lower_border = lb_interactions
    upper_border = upb_ineractions
    item_border = top_items

    events = data.copy()

    # Приведем к временному формату данные из timestapm
    events["timestamp"] = pd.to_datetime(events["timestamp"], unit="ms")
    events.rename(columns={"timestamp": "date"}, inplace=True)
    events["date"] = pd.to_datetime(events["date"].dt.date)
    events = events.sort_values("date").reset_index(drop=True)
    events = events[["visitorid", "itemid", "event", "date"]]

    # Фильтруем по количеству больше 30 и оставляем только itemid и count
    top_items = pd.DataFrame(
        events.groupby("itemid")["event"].value_counts()
    ).reset_index()
    top_items = top_items[top_items["count"] >= item_border][["itemid", "count"]]
    top_items = top_items.groupby("itemid")["count"].sum().to_dict()

    # Создаем новый столбец num_occur, в котором хранится количество событий для каждого itemid
    events["num_occur"] = events["itemid"].map(top_items)

    # Фильтруем события, оставляем только те, у которых num_occur больше 1000
    events_processed = events[
        (events["num_occur"] >= lower_border) & (events["num_occur"] < upper_border)
    ]
    events_processed = events_processed.drop(columns="num_occur")
    events_processed = events_processed.drop_duplicates().reset_index(drop=True)

    # Выделяем обучающий набор данных до сентября
    events_train = events_processed[events_processed["date"].dt.month < 8]
    # Выделяем тестовый набор данных с сентября и позже
    events_test = events_processed[events_processed["date"].dt.month >= 8]

    # Фильтруем тестовый набор данных
    events_test = events_test[
        (events_test["visitorid"].isin(events_train["visitorid"]))
        & (events_test["itemid"].isin(events_train["itemid"]))
    ]

    # Список категориальных признаков
    id_cols = ["visitorid", "itemid"]

    # Создаем словарь для закодированных значений обучающего набора
    trans_cat_train = dict()
    # Создаем словарь для закодированных значений тестового набора
    trans_cat_test = dict()

    for k in id_cols:
        cate_enc = LabelEncoder()
        trans_cat_train[k] = cate_enc.fit_transform(
            events_train[k].values
        )  # Кодируем значения обучающего набора
        trans_cat_test[k] = cate_enc.transform(
            events_test[k].values
        )  # Кодируем значения тестового набора

    # Создаем словарь для закодированных значений целевой переменной
    ratings = dict()

    cate_enc_2 = LabelEncoder()
    ratings["train"] = cate_enc_2.fit_transform(
        events_train.event
    )  # Кодируем целевую переменную для обучающего набора
    ratings["test"] = cate_enc_2.transform(
        events_test.event
    )  # Кодируем целевую переменную для тестового набора

    # Вычисляем количество уникальных пользователей
    n_users = len(np.unique(trans_cat_train["visitorid"]))
    # Вычисляем количество уникальных товаров
    n_items = len(np.unique(trans_cat_train["itemid"]))

    # Создаем словарь для матриц оценок
    rate_matrix = dict()

    # Создаем разреженную матрицу для обучающего набора
    rate_matrix["train"] = coo_matrix(
        (ratings["train"], (trans_cat_train["visitorid"], trans_cat_train["itemid"])),
        shape=(n_users, n_items),
    )
    # Создаем разреженную матрицу для тестового набора
    rate_matrix["test"] = coo_matrix(
        (
            ratings["test"],  # данные
            (trans_cat_test["visitorid"], trans_cat_test["itemid"]),
        ),  # индексы строк (trans_cat_test[“visitorid”]) и индексы столбцов (trans_cat_test[“itemid”])
        shape=(n_users, n_items),
    )

    # Создаем модель LightFM с указанием параметров
    model = LightFM(no_components=10, loss="warp")
    # Обучаем модель на обучающей матрице
    model.fit(rate_matrix["train"], epochs=100, num_threads=8)

    # Вычисляем среднюю точность на тестовой матрице для k=3
    map_at3 = precision_at_k(model, rate_matrix["test"], k=3).mean()
    print(f"Mean Average Precision at 3: {round(map_at3*100, 3)} %")

    # Используем обученную модель для предсказания предпочтений пользователей для товаров в тестовой выборке
    predicted_scores = model.predict(
        trans_cat_test["visitorid"], trans_cat_test["itemid"], num_threads=8
    )

    # Создаем функцию для получения предсказанных оценок для всех возможных пар (пользователь, товар) в тестовой выборке
    def get_predicted_ratings(visitor_ids, item_ids, scores):
        predicted_ratings = pd.DataFrame(
            {"visitorid": visitor_ids, "itemid": item_ids, "predicted_score": scores}
        )
        return predicted_ratings

    # Преобразуем полученные предсказанные оценки в датафрейм с колонками visitorid и itemid
    predicted_ratings_df = get_predicted_ratings(
        events_test["visitorid"], events_test["itemid"], predicted_scores
    )

    predicted_ratings_df = predicted_ratings_df[
        predicted_ratings_df["predicted_score"] < 0
    ]

    predicted_ratings_pivot = pd.pivot_table(
        data=predicted_ratings_df,
        index="visitorid",
        columns="itemid",
        values="predicted_score",
        aggfunc="sum",
    )

    users = predicted_ratings_pivot.index.to_list()
    items = predicted_ratings_pivot.columns.to_list()
    scores = np.array(predicted_ratings_pivot)

    rec_list = []
    for i in range(len(users)):

        var_list = []
        best_var = np.argsort(scores[i])[:3].tolist()
        for j in best_var:
            var_list.append(items[j])

        # print({users[i]: var_list})
        rec_list.append(var_list)

    recomendations = pd.DataFrame(data={"users": users, "recomendations": rec_list})

    return recomendations

In [ ]:
# Формируем 3 основные группы пользователей
random_customers = rs_batch(
    data=events_raw, lb_interactions=0, upb_ineractions=100, top_items=4
)
temp_customers = rs_batch(
    data=events_raw, lb_interactions=500, upb_ineractions=1000, top_items=4
)
mvp_customers = rs_batch(
    data=events_raw, lb_interactions=1000, upb_ineractions=100000, top_items=4
)

In [ ]:
# Почистим дублирующие рекомендации для юзеров с приоритетом MVP->Temp->Random
temp_customers = temp_customers[~temp_customers["users"].isin(mvp_customers["users"])]

random_customers = random_customers[
    ~random_customers["users"].isin(mvp_customers["users"])
]
random_customers = random_customers[
    ~random_customers["users"].isin(temp_customers["users"])
]

In [ ]:
# Сформируем датафрейм с рекомендациями для пользователей проявивших хоть какую-то активность
rec_df = pd.concat([random_customers, temp_customers, mvp_customers]).reset_index(
    drop=True
)
rec_df

In [ ]:
# Сформируем рекомендации для оставшихся
# 
unique_usrs = events_raw["visitorid"].unique().tolist()